_<span style="color: var(--vscode-foreground);">CREATE - Dotazy, které něco stvořily</span>  
<span style="color: var(--vscode-foreground);">INFO - Dotazy, kterými jsme jen něco zjišťovaly</span>_

### <u>Záloha tabulky</u>

In [ ]:
-- Kopie tabulky (záloha)                                                                                                       CREATE
SELECT *
INTO Catastrophes_backup
FROM [dbo].[Catastrophes new];

### <u>Ověření, zda může být identifikátor použe číselná část z DisNo - NEMŮŽE</u>

In [ ]:
-- Chceme vySELECTovat ze sloupce DisNo pouze číselnou část: 1967-0026                                                          INFO
SELECT count(distinct((LEFT(DisNo, 9)))) AS PK_count_distinct 
FROM [dbo].[Catastrophes new];

SELECT count(LEFT(DisNo, 9)) AS PK_count
FROM [dbo].[Catastrophes new];
-- -> nemůžeme vzít pouze číselnou část sloupce jako PK, protože není unikátní 
-- (při použítí count a count(distinct) se nerovnají výsledky 24294 != 26188)

### <u>Nová tabulka Location\_table.</u>

In [ ]:
-- Jsou nějaké country, co mají více členů?                                                                                      INFO
-- -> ANO, DVĚ - DO TABULKY LOKACÍ DÁT I COUNTRY [POPŘ. REGION, SUBREGION (ale ty sou konzistentní - není tam více záznamů oddělených čárkou)]
SELECT DISTINCT(country)
FROM [dbo].[Catastrophes new]
WHERE Country LIKE '%,%';

In [11]:
-- Vytvoření Nové tabulky s lokacemi Location:                                                                            CREATE
CREATE TABLE Location (
    DisNo NVARCHAR(100),
    Location NVARCHAR(MAX),
    Dirty_location_separator bit,
    FOREIGN KEY (DisNo) REFERENCES [dbo].[Catastrophes new] (DisNo)
);

Commands completed successfully.

Total execution time: 00:00:00.052

In [3]:
-- Vložení DisNo a Location do nové tabulky "Location" a rozdělení podle "," do nových řádků                                CREATE
--BEGIN TRANSACTION;
INSERT INTO Location (DisNo, Location) 
SELECT DisNo, value AS Location
FROM [Catastrophes new]
CROSS APPLY STRING_SPLIT(Location, ',')
WHERE Location NOT LIKE N'%;%'
--COMMIT;

(70415 rows affected)

Total execution time: 00:00:10.397

In [5]:
-- Vložení a rozdělení řádků podle ";". Tam, kde je ; přidání 1 do sloupce Dirty_location_separator.                          CREATE
INSERT INTO Location (DisNo, Location, Dirty_location_separator)
SELECT DisNo, value AS Location, 1 AS Dirty_location_separator
FROM [Catastrophes new]
CROSS APPLY STRING_SPLIT(Location, ';')
WHERE Location LIKE N'%;%';

(1711 rows affected)

Total execution time: 00:00:00.978

In [9]:
-- Odstranění bílých znaků z tabulky "Location".                                                                 CREATE
UPDATE Location
SET Location = LTRIM(RTRIM(Location));

(72126 rows affected)

Total execution time: 00:00:10.268

In [ ]:
--1337 řádků, které obsahují N'% and %' - rozhodli jsme se, že zanedbáme.                                           INFO!
SELECT *
FROM Location
WHERE Location LIKE N'% and %' 

In [15]:
-- Smazání sloupce Location z Catastrophes new.                                                                     CREATE
ALTER TABLE "Catastrophes new" DROP COLUMN "Location";


Commands completed successfully.

Total execution time: 00:00:00.075

### Kontrola

In [ ]:
-- Snaha zjistit k jakému roku je sloupec "Total Damage, Adjusted" (navýšený o inflaci) vztažený.                        INFO
--> v letech 2022 se podíl rovná 1 tzn., že inflace je vztážená k roku generování datasetu.
SELECT Country, Start_Year, (Total_Damage_000_US / NULLIF(Total_Damage_Adjusted_000_US, 0)) as neco
FROM [dbo].[Catastrophes new]
WHERE (Total_Damage_000_US / NULLIF(Total_Damage_Adjusted_000_US, 0)) is not null 
ORDER BY Country, Start_Year;


SELECT Country, Start_Year, (Total_Damage_000_US / NULLIF(Total_Damage_Adjusted_000_US, 0)) as neco
FROM [dbo].[Catastrophes new]
WHERE (Total_Damage_000_US / NULLIF(Total_Damage_Adjusted_000_US, 0)) is not null and Start_Year = '2022'
ORDER BY Country, Start_Year;


In [1]:
-- Kontrola totožného počtu řádku se zdrojovým souborem (v csv 26188). - SEDÍ                                                 INFO
SELECT count(*) as Pocet_radku
FROM [dbo].[Catastrophes new];


In [ ]:
-- Snaha o porovnání konzistence záznamů v ISO a Country (že není v Country jednou United States of America a                  INFO
-- jednou třeba United States, porovnání počtu ISO a názvu v Country?)
-- pouze USA: 1498
SELECT count(*)
FROM [dbo].[Catastrophes new]
WHERE ISO = 'USA'
-- pouze USA: 1498 - SEDÍ
SELECT count(*)
FROM [dbo].[Catastrophes new]
WHERE Country = 'United States of America';

In [ ]:
-- Konrola konzistence sloupců ISO a Country. - SEDÍ                                                                             INFO
SELECT DISTINCT ISO, Country
FROM [dbo].[Catastrophes new]
ORDER BY ISO, Country ASC;

In [ ]:
-- TOP 10 zemí podle počtu katastrof.                                                                                           INFO
SELECT TOP 10 Country, COUNT([Disaster_Type]) AS CountType
FROM [dbo].[Catastrophes new]
GROUP BY Country
ORDER BY CountType DESC;

In [ ]:
-- TOP 10 zemí podle počtu zemřelých.   
                                                                                          INFO
SELECT TOP 10 Country, SUM([Total_Deaths]) AS TotalDeaths
FROM [dbo].[Catastrophes new]
GROUP BY Country
ORDER BY TotalDeaths DESC;



In [ ]:
-- TOP 10 zemí podle škod.                                                                                                          INFO
SELECT TOP 10 Country, SUM([Total_Damage_Adjusted_000_US]) AS SumTotalDamage
FROM [dbo].[Catastrophes new]
GROUP BY Country
ORDER BY SumTotalDamage DESC;